In [257]:
from pynq import Xlnk
from pynq import Overlay
import numpy as np
import pynq.lib.dma
import time

overlay = Overlay('D1.bit')
dma = overlay.const_multiply.multiply_dma

xlnk = Xlnk()



In [274]:
class Convolution2D:
    def __init__(self,Input_planes,Out_planes,Conv_H,Conv_W,Stride_H, Stride_W, Padding,Relu,Weight, Bias=np.any([False]),Precision=10000):
        self.Input_planes=Input_planes
        self.Out_planes=Out_planes
        self.Conv_H=Conv_H
        self.Conv_W=Conv_W
        self.Stride_H=Stride_H
        self.Stride_W=Stride_W 
        self.Padding=Padding
        self.Relu=Relu
        assert len(Weight.shape)==4, 'Input Tensor deminision should be (D,X,Y,Z) '
        self.Weight=np.multiply(Weight.flatten(),Precision)
        if (Bias.any()):
            self.Bias=np.multiply(Bias,Precision)
        else:
            self.Bias=np.any([False])
        self.Precision=Precision # precision of weights and bias in our calculation inside FPGA
    def forward(self,data,dma,Load_Input=1,Load_Weight=1):
        
        assert len(data.shape)==3, 'Input Tensor deminision should be (X,Y,Z) '
        in_buffer = xlnk.cma_array(shape=(3000,), dtype=np.int32)
        data=np.multiply(data,self.Precision)
        x=data.flatten()
        size=data.shape
        
        
        Configuration= xlnk.cma_array(shape=(17,), dtype=np.int32)
        Configuration[0]=0
        Configuration[1]=x.shape[0]
        Configuration[2]=size[0] #image deminsion d
        Configuration[3]=size[1] #image deminsion h
        Configuration[4]=size[2] #image deminsion w
        Configuration[5]=self.Out_planes # Number of filters
        Configuration[6]=self.Input_planes # first deminsion of weights
        Configuration[7]=self.Conv_H
        Configuration[8]=self.Conv_W
        Configuration[9]=self.Stride_H
        Configuration[10]=self.Stride_W
        Configuration[11]=self.Padding
        if (self.Bias.any()):
            Configuration[12]=1
        else:
            Configuration[12]=0  
        Configuration[13]=self.Relu
        Configuration[14]=self.Precision
        Configuration[15]=Load_Input
        Configuration[16]=Load_Weight
        dma.sendchannel.transfer(Configuration)
        dma.sendchannel.wait()
        dma.recvchannel.transfer(Configuration)
        dma.recvchannel.wait()

        
        # upload image inside FPGA
        for idx in range(int((x.shape[0])/3000)):
            temproray= x[(idx*3000):(idx*3000)+3000]
            
            np.copyto(in_buffer,temproray,casting='unsafe')
            dma.sendchannel.transfer(in_buffer)
            dma.sendchannel.wait()
            dma.recvchannel.transfer(in_buffer)
            dma.recvchannel.wait()
        if((x.shape[0])%3000!=0):
            in_buffer_remind=xlnk.cma_array(shape=(((x.shape[0])%3000),), dtype=np.int32)
            temproray= x[int((x.shape[0])/3000)*3000:x.shape[0]]
            np.copyto(in_buffer_remind,temproray,casting='unsafe')
            dma.sendchannel.transfer(in_buffer_remind)
            dma.sendchannel.wait()
            dma.recvchannel.transfer(in_buffer_remind)
            dma.recvchannel.wait()
      
        
        # upload Bias inside FPGA
        
        if(self.Bias.any()):
            
            for idx in range(int((self.Bias.shape[0])/3000)):
                temproray= self.Bias[(idx*3000):(idx*3000)+3000]
                np.copyto(in_buffer,temproray,casting='unsafe')
                dma.sendchannel.transfer(in_buffer)
                dma.sendchannel.wait()
                dma.recvchannel.transfer(in_buffer)
                dma.recvchannel.wait()
            if((self.Bias.shape[0])%3000!=0):
                in_buffer_remind=xlnk.cma_array(shape=(((self.Bias.shape[0])%3000),), dtype=np.int32)
                temproray= self.Bias[int((self.Bias.shape[0])/3000)*3000:self.Bias.shape[0]]
                np.copyto(in_buffer_remind,temproray,casting='unsafe')
                dma.sendchannel.transfer(in_buffer_remind)
                dma.sendchannel.wait()
                dma.recvchannel.transfer(in_buffer_remind)
                dma.recvchannel.wait()
        
        # upload weights inside FPGA
        
        
        for idx in range(int((self.Weight.shape[0])/3000)):
            temproray= self.Weight[(idx*3000):(idx*3000)+3000]
            np.copyto(in_buffer,temproray,casting='unsafe')
            dma.sendchannel.transfer(in_buffer)
            dma.sendchannel.wait()
            dma.recvchannel.transfer(in_buffer)
            dma.recvchannel.wait()
        if((self.Weight.shape[0])%3000!=0):
            in_buffer_remind=xlnk.cma_array(shape=(((self.Weight.shape[0])%3000),), dtype=np.int32)
            temproray= self.Weight[int((self.Weight.shape[0])/3000)*3000:self.Weight.shape[0]]
            np.copyto(in_buffer_remind,temproray,casting='unsafe')
            dma.sendchannel.transfer(in_buffer_remind)
            dma.sendchannel.wait()
            dma.recvchannel.transfer(in_buffer_remind)
            dma.recvchannel.wait()

        # Get final Result size
        in_buffer_remind=xlnk.cma_array(shape=(3,), dtype=np.int32)
        dma.sendchannel.transfer(in_buffer_remind)
        dma.sendchannel.wait()
        dma.recvchannel.transfer(in_buffer_remind)
        dma.recvchannel.wait() 
        
        Total_result=in_buffer_remind[0]
        Out_H =in_buffer_remind[1]
        Out_W =in_buffer_remind[2]
        Output=np.zeros((Total_result,))
        

        # Return Final size from FPGA
        for idx in range(int((Total_result)/3000)):
            dma.sendchannel.transfer(in_buffer)
            dma.sendchannel.wait()
            dma.recvchannel.transfer(in_buffer)
            dma.recvchannel.wait()
            np.copyto(Output[(idx*3000):((idx*3000)+3000)],in_buffer,casting='unsafe')
        if((Total_result)%3000!=0):
            in_buffer_remind=xlnk.cma_array(shape=(((Total_result)%3000),), dtype=np.int32)
            dma.sendchannel.transfer(in_buffer_remind)
            dma.sendchannel.wait()
            dma.recvchannel.transfer(in_buffer_remind)
            dma.recvchannel.wait()
            np.copyto(Output[(int((Total_result)/3000)*3000):Total_result],in_buffer_remind,casting='unsafe')
            
        # Return result to float format and reshape
        Output=np.divide(Output,self.Precision)#.reshape(self.Out_planes,Out_H,Out_W)
        return Output
        

In [275]:
conv1_w=np.load('Conv1_weight.np.npy')
conv1_b=np.load('Conv1_bias.np.npy')
conv2_w=np.load('Conv2_weight.np.npy')
conv2_b=np.load('Conv2_bias.np.npy')
f1_w=np.load('fc1_weight.np.npy')
f2_w=np.load('fc2_weight.np.npy')
f1_b=np.load('fc1_bias.np.npy')
f2_b=np.load('fc2_bias.np.npy')
data=np.load('mnist_data.npy')
target1=np.load('mnist_label.npy')
#Input_planes,Out_planes,Conv_H,Conv_W,Stride_H, Stride_W, Padding,Relu,Weight, Bias=np.any([False]),Precision=10000):


In [276]:
'''
for i in range(10):
    conv1_b[i]=1
    for i2 in range(5):
        for i3 in range(5):
            conv1_w[i][0][i2][i3]=1 
'''
conv1=Convolution2D(1,10,5,5,1,1,0,0,conv1_w,conv1_b,10000000)



In [284]:
total = 0
correct = 0
data2 = data[0:3]
target=target1[0:3]
size = data2.shape
#CONV1 (10,5,5)
rs1 = np.zeros((size[0], 10, 24, 24))
for i0 in range(0, size[0]):
    for i in range(0, 10):
        for i1 in range(0, 24):
            for i2 in range(0, 24):
                tmp = data2[i0, 0, i1:i1 + 5, i2:i2 + 5]
                rs1[i0][i][i1][i2] = np.sum(np.multiply(tmp, conv1_w[i])) + conv1_b[i]

In [285]:
rs2=conv1.forward(data2[0],dma)


In [288]:
#rs1=rs1[0]
#rs1=rs1.flatten()
#print(rs1.shape,rs2.shape)
for i in range(5760):
    print(i,rs2[i],rs1[i])

0 -0.0206547 -0.0206545963883
1 -0.0206547 -0.0206545963883
2 -0.0206547 -0.0206545963883
3 -0.0206547 -0.0206545963883
4 -0.0206547 -0.0206545963883
5 -0.0206547 -0.0206545963883
6 -0.0206547 -0.0206545963883
7 -0.0206547 -0.0206545963883
8 -0.0206547 -0.0206545963883
9 -0.0206547 -0.0206545963883
10 -0.0206547 -0.0206545963883
11 -0.0206547 -0.0206545963883
12 -0.0206547 -0.0206545963883
13 -0.0206547 -0.0206545963883
14 -0.0206547 -0.0206545963883
15 -0.0206547 -0.0206545963883
16 -0.0206547 -0.0206545963883
17 -0.0206547 -0.0206545963883
18 -0.0206547 -0.0206545963883
19 -0.0206547 -0.0206545963883
20 -0.0206547 -0.0206545963883
21 -0.0206547 -0.0206545963883
22 -0.0206547 -0.0206545963883
23 -0.0206547 -0.0206545963883
24 -0.0206547 -0.0206545963883
25 -0.0206547 -0.0206545963883
26 -0.0206547 -0.0206545963883
27 -0.0206547 -0.0206545963883
28 -0.0206547 -0.0206545963883
29 -0.0206547 -0.0206545963883
30 -0.0206547 -0.0206545963883
31 -0.0206547 -0.0206545963883
32 -0.0206547 -0.0

305 0.243649 0.243648841977
306 -0.9754531 -0.975453495979
307 -2.2822514 -2.28225135803
308 -1.9838602 -1.98386061192
309 -1.3368514 -1.33685147762
310 -0.4189074 -0.418907403946
311 -0.0206547 -0.0206545963883
312 -0.0206547 -0.0206545963883
313 -1.1406591 -1.14065921307
314 -2.306614 -2.30661487579
315 -3.1464962 -3.14649748802
316 -3.0662716 -3.0662727356
317 -1.4025147 -1.40251481533
318 -0.308215 -0.308214992285
319 0.1189538 0.118953719735
320 -0.238852 -0.238852292299
321 -0.8063053 -0.806305646896
322 -1.071148 -1.07114803791
323 -1.1098771 -1.10987699032
324 -1.6738007 -1.67380094528
325 -1.4605204 -1.46052098274
326 -0.3996963 -0.399697184563
327 1.2506686 1.25066876411
328 3.1070444 3.10704517365
329 3.8423048 3.84230494499
330 3.3487188 3.34871912003
331 1.7040974 1.70409762859
332 0.6538108 0.653810977936
333 0.0562444 0.0562444329262
334 -0.1038871 -0.103887170553
335 -0.0206547 -0.0206545963883
336 -0.0206547 -0.0206545963883
337 -0.7045486 -0.704548597336
338 -1.762121

623 -0.1125117 -0.229252502322
624 -0.1125117 -0.229252502322
625 -0.1125117 -0.229252502322
626 -0.1125117 -0.229252502322
627 -0.1125117 -0.229252502322
628 -0.1125117 -0.229252502322
629 -0.139594 -0.262968480587
630 -0.3293666 -0.378454208374
631 -0.8271265 -0.406681835651
632 -1.5638164 -0.214545443654
633 -2.4586022 0.0906008780003
634 -3.1661524 0.612564146519
635 -3.2170032 0.958953917027
636 -2.7789174 0.966545522213
637 -1.9849488 0.738520085812
638 -1.0836991 0.340859591961
639 -0.5227653 0.00785697251558
640 -0.2516082 -0.128035008907
641 -0.1125117 -0.229252502322
642 -0.1125117 -0.229252502322
643 -0.1125117 -0.229252502322
644 -0.1125117 -0.229252502322
645 -0.1125117 -0.229252502322
646 -0.1125117 -0.229252502322
647 -0.1125117 -0.229252502322
648 -0.1125117 -0.229252502322
649 -0.1125117 -0.229252502322
650 -0.1125117 -0.229252502322
651 -0.1125117 -0.229252502322
652 -0.1125117 -0.229252502322
653 -0.1275015 -0.491298854351
654 -0.216926 -1.05118644238
655 -0.3340188 

1000 -2.7870242 0.916804730892
1001 -2.6024516 -0.260325968266
1002 -1.8933646 0.368410050869
1003 -1.0474329 1.01810467243
1004 -0.307822 0.770281910896
1005 -0.0246436 0.145723894238
1006 -0.0842078 -0.178206294775
1007 -0.1125117 -0.229252502322
1008 -0.1125117 -0.229252502322
1009 -0.1125117 -0.229252502322
1010 -0.1125117 -0.229252502322
1011 -0.1125117 -0.229252502322
1012 -0.1125117 -0.229252502322
1013 -0.1125117 -0.229252502322
1014 -0.6585426 -0.517160475254
1015 -1.2891235 -1.33873617649
1016 -1.936004 -1.95471048355
1017 -2.328167 -3.93036532402
1018 -1.9408396 -5.99205827713
1019 -1.3249134 -5.76814699173
1020 -0.6447337 -0.322629332542
1021 0.1061431 5.92270708084
1022 0.2805887 7.8158326149
1023 0.3684181 4.7787027359
1024 0.2264582 1.27663648129
1025 -0.0638289 -0.141453027725
1026 -0.1125117 -0.229252502322
1027 -0.1125117 -0.229252502322
1028 -0.1125117 -0.229252502322
1029 -0.1125117 -0.229252502322
1030 -0.1125117 -0.229252502322
1031 -0.1125117 -0.229252502322
1032

1295 -0.0995305 -0.276158034801
1296 -0.0995305 -0.276158034801
1297 -0.0995305 -0.276158034801
1298 -0.0995305 -0.276158034801
1299 -0.0170595 -1.17565214634
1300 0.1394923 -2.63505673409
1301 0.1030024 -2.47264027596
1302 -0.1032416 1.04907369614
1303 -0.2976886 5.51116466522
1304 -0.7981159 6.8004617691
1305 -1.3257142 3.60466384888
1306 -1.817129 0.433568179607
1307 -1.725261 -0.723384141922
1308 -0.950851 -0.503503918648
1309 0.8336637 -1.19192826748
1310 2.43306 -1.39366292953
1311 3.6739892 -0.0286388248205
1312 3.715058 2.21494913101
1313 2.3837358 2.00631165504
1314 1.2328724 0.0599282681942
1315 0.2008104 -0.74156075716
1316 -0.0320715 -0.357098698616
1317 -0.0995305 -0.276158034801
1318 -0.0995305 -0.276158034801
1319 -0.0995305 -0.276158034801
1320 -0.0995305 -0.276158034801
1321 -0.1031903 -0.287443220615
1322 -0.2290734 -0.640321075916
1323 -0.0374985 -2.41905713081
1324 0.5692516 -2.85097956657
1325 0.1187153 0.197421446443
1326 -0.2226704 4.55413341522
1327 -0.9240401 6

...

4163 4.3735752 -5.32998275757
4164 5.3904332 -4.58169317245
4165 6.171876 -2.92106151581
4166 6.0880008 -0.579250752926
4167 5.2514372 2.26913285255
4168 3.4456872 3.81270027161
4169 1.1196998 3.66188240051
4170 -0.1254926 2.54327821732
4171 -0.4893398 0.945968270302
4172 -0.0008982 0.0623709894717
4173 0.0646187 -0.0141098573804
4174 0.0646187 -0.0141098573804
4175 0.0646187 -0.0141098573804
4176 0.0646187 -0.0141098573804
4177 0.0646187 -0.0141098573804
4178 0.0646187 -0.0141098573804
4179 0.1470897 0.429664194584
4180 0.3036416 2.24363040924
4181 0.2671517 2.99038124084
4182 0.0609076 2.91998553276
4183 -0.1335393 0.0951950997114
4184 -0.6339666 -2.07296013832
4185 -1.1615649 -3.22513723373
4186 -1.6529797 -3.75436520576
4187 -1.5611118 -3.95932817459
4188 -0.7867017 -4.69318056107
4189 0.997813 -4.88994455338
4190 2.5972092 -3.72048139572
4191 3.8381384 -1.86320817471
4192 3.8792072 -0.0191076099873
4193 2.5478852 0.644100427628
4194 1.3970217 0.867317438126
4195 0.3649597 0.676620

4488 0.0646187 -0.0141098573804
4489 0.0646187 -0.0141098573804
4490 0.0646187 -0.0141098573804
4491 0.0646187 -0.0141098573804
4492 0.0646187 -0.0141098573804
4493 0.0646187 -0.0141098573804
4494 0.0646187 -0.0141098573804
4495 0.0463198 -0.00599379837513
4496 0.0660718 0.396998614073
4497 -0.25115 1.02701544762
4498 -0.5735365 1.37110257149
4499 -0.4243501 0.06868814677
4500 0.3996028 -1.32722532749
4501 1.5419706 -0.724852204323
4502 1.9901582 -0.0597298964858
4503 1.790222 0.403171211481
4504 0.5261018 0.308764129877
4505 0.0646187 -0.0141098573804
4506 0.0646187 -0.0141098573804
4507 0.0646187 -0.0141098573804
4508 0.0646187 -0.0141098573804
4509 0.0646187 -0.0141098573804
4510 0.0646187 -0.0141098573804
4511 0.0646187 -0.0141098573804
4512 0.0646187 -0.0141098573804
4513 0.0646187 -0.0141098573804
4514 0.0646187 -0.0141098573804
4515 0.0646187 -0.0141098573804
4516 0.0646187 -0.0141098573804
4517 0.0646187 -0.0141098573804
4518 0.0646187 -0.0141098573804
4519 0.07728 0.1137057095

4869 -0.0594554 0.0317001380026
4870 -0.0594554 0.0317001380026
4871 -0.0594554 0.0317001380026
4872 -0.0594554 0.0317001380026
4873 0.3836344 1.78086781502
4874 0.846904 4.01870393753
4875 1.5566756 3.14634919167
4876 1.6024963 1.87640202045
4877 0.8832728 0.756886601448
4878 1.0021502 -0.426873266697
4879 0.5763935 -0.0944258868694
4880 0.4961075 -0.511883914471
4881 0.3821299 -0.135159492493
4882 0.0175312 0.0424462892115
4883 -0.0929784 0.268254905939
4884 -0.2804315 2.43913888931
4885 -0.6141799 2.76008749008
4886 -0.9642342 0.304904967546
4887 -1.2910844 -2.88680076599
4888 -1.5622754 -4.064889431
4889 -1.7758272 -3.30453300476
4890 -1.8181136 -2.92322134972
4891 -1.7994844 -1.60487639904
4892 -1.2481974 -0.949729025364
4893 -0.524772 -0.543961524963
4894 -0.0865962 -0.00399780087173
4895 -0.0594554 0.0317001380026
4896 -0.0594554 0.0317001380026
4897 -0.5500242 1.52022218704
4898 -1.2062883 3.8182425499
4899 -0.9880182 3.50700044632
4900 -0.9792577 1.98860967159
4901 -0.0133536 

5171 0.2664421 -0.378901571035
5172 1.0763749 -0.407140374184
5173 0.9342898 -0.68782377243
5174 0.5418851 -0.0708014369011
5175 -0.0288871 0.0359669886529
5176 -0.0594554 0.0317001380026
5177 -0.0594554 0.0317001380026
5178 -0.0594554 0.0317001380026
5179 -0.0594554 0.0317001380026
5180 -0.0594554 0.0317001380026
5181 -0.0594554 0.0317001380026
5182 -0.0594554 0.0317001380026
5183 -0.0594554 0.0317001380026
5184 -0.062626 0.224871993065
5185 -0.062626 0.224871993065
5186 -0.062626 0.224871993065
5187 -0.062626 0.224871993065
5188 -0.062626 0.224871993065
5189 -0.062626 0.224871993065
5190 -0.062626 0.224871993065
5191 -0.062626 0.224871993065
5192 -0.062626 0.224871993065
5193 -0.062626 0.224871993065
5194 -0.062626 0.224871993065
5195 -0.062626 0.224871993065
5196 -0.062626 0.224871993065
5197 -0.062626 0.224871993065
5198 -0.062626 0.224871993065
5199 -0.062626 0.224871993065
5200 -0.062626 0.224871993065
5201 -0.062626 0.224871993065
5202 -0.062626 0.224871993065
5203 -0.062626 0.2

5548 -0.3216349 0.486106902361
5549 -0.2275408 0.224016517401
5550 0.3418434 0.00492381304502
5551 1.2158149 0.113150477409
5552 2.2997334 0.583940148354
5553 3.78612 0.969880402088
5554 4.9054204 0.43777987361
5555 4.3228016 -0.360794574022
5556 3.0232696 -1.01633238792
5557 1.3086073 -0.559169590473
5558 -0.152335 0.15328194201
5559 0.1653942 0.73597997427
5560 0.7026474 1.53106665611
5561 1.2408392 2.08735871315
5562 1.899296 1.0493683815
5563 2.0362304 -0.564544737339
5564 1.7776382 -1.71101796627
5565 1.1806018 -1.14152932167
5566 0.2962817 -0.0520228222013
5567 -0.062626 0.224871993065
5568 -0.062626 0.224871993065
5569 -0.062626 0.224871993065
5570 -0.062626 0.224871993065
5571 -0.062626 0.224871993065
5572 -0.062626 0.224871993065
5573 -0.062626 0.224871993065
5574 0.0194749 0.23472225666
5575 0.1621035 0.783147335052
5576 0.3240293 1.84263849258
5577 0.5433346 2.88079452515
5578 0.9518627 2.7703499794
5579 1.0734502 1.9323823452
5580 0.6103878 0.354492306709
5581 -0.4281972 -0

In [178]:
#POOLING
rs2 = np.zeros((size[0], 10, 12, 12))
for i0 in range(0, size[0]):
    for i in range(0, 10):
        i1=0
        while i1<24:
            i2=0
            while i2<24:
                tmp = rs[i0, i, i1:i1 + 2, i2:i2 + 2]
                tmp=np.max(tmp)
#RELU
                if tmp>0:
                    rs2[i0][i][int(i1/2)][int(i2/2)] =tmp
                i2+=2
            i1+=2

#conv2 (20,5,5)
rs = np.zeros((size[0], 20, 8, 8))
for i0 in range(0, size[0]):
    for i in range(0, 20):
        for i1 in range(0, 8):
            for i2 in range(0, 8):
                tmp = rs2[i0,:, i1:i1 + 5, i2:i2 + 5]
                rs[i0][i][i1][i2] = np.sum(np.multiply(tmp, conv2_w[i])) + conv2_b[i]
#POOLING
rs2 = np.zeros((size[0], 20, 4, 4))
for i0 in range(0, size[0]):
    for i in range(0, 20):
        i1=0
        while i1<8:
            i2=0
            while i2<8:
                tmp = rs[i0, i, i1:i1 + 2, i2:i2 + 2]
                tmp=np.max(tmp)
#Relu
                if tmp>0:
                    rs2[i0][i][int(i1/2)][int(i2/2)] =tmp
                i2+=2
            i1+=2
#Flatteren
rs = np.zeros((size[0],320))
#FC1 320 --->50
for i0 in range(0, size[0]):
    tmp=rs2[i0]
    rs[i0]=tmp.flatten()
rs2 = np.zeros((size[0], 50))
for i0 in range(0,size[0]):
    tmp=rs[i0]
    tmp=np.add(np.dot(f1_w, tmp), f1_b)
    for i in range(0,50):
#RELU
        if tmp[i]>0:
            rs2[i0][i]=tmp[i]
#FC2 50 ---> 10
rs = np.zeros((size[0], 10))
for i0 in range(0,size[0]):
    tmp=rs2[i0]
    rs[i0]=np.add(np.dot(f2_w, tmp), f2_b)
# LOG SOFTMAX
rs=rs
for i in range(0, size[0]):
    if np.argmax(rs[i])==target[i]:
        #print(np.argmax(rs[i]),target[i])
        correct+=1
    total+=1
print ('accuracy=',float(correct)/float(total))

(10, 24, 24)